In [1]:
import os
import csv
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [2]:
working_dir = os.path.expanduser("~/projects/NLP/NLP-language-detection/data/clean_data")

In [3]:
# Load train and test set from text
train_df = pd.read_csv(f"{working_dir}/train.txt", sep="\t", header=None, names=["text", "label"], quoting=csv.QUOTE_NONE)
test_df = pd.read_csv(f"{working_dir}/test.txt", sep="\t", header=None, names=["text"], quoting=csv.QUOTE_NONE)
test_labels = pd.read_csv(f"{working_dir}/test_labels.txt", header=None, names=["label"], quoting=csv.QUOTE_NONE)

In [4]:
# Visualize train data
train_df

,text,label
0,000 ألف قنطار من الأعلاف برسم الموسم الفلاحي ا...,ar
1,000 برميل يوميا من طرف البلدان المنتجة خارجها.,ar
2,000 برميل يوميا من طرف بلدان الأوبيب و400.,ar
3,000 توقيع فردي على الأقل لناخبين مسجلين في قائ...,ar
4,000 دج للتر الواحد، حسب محمد عبد الجبار.,ar
...,...,...
1679995,ພວກເຂາເຈາໄດນຳໃຊເຄອງຈກດງກາວຢາງສຳເລດຜນ.,lo
1679996,ເລມດມນຳໃນຕອນເຊາໃນມທດສອບ ຫ ຢາງໜອຍ 2 ຊວໂມງກອນເກບ...,lo
1679997,"ເພອໃຫໄດຮບການຕອບຮບ, ໜງໃນວທທດຄການບອກອກຝາຍໜງວາເຈາ...",lo
1679998,ປບປງຕເສອຜາ.,lo


In [5]:
# Visualize test data
test_df

,text
0,وقالت السعيدي: «أنا مناضلة بصيغة فنية، تحرص عل...
1,وقالت السفارة الإسرائيلية في روسيا إن وزارة ال...
2,وقالت السفارة الأمريكية إنها ستستمر في تقديم ت...
3,وقالت السفارة الأمريكية، عبر “تويتر” في 16 نوف...
4,"وقالت السفارة الأمريكية عبر ""تويتر"": ""منزعجون ..."
...,...
419995,ພາຍຫງໄດກເຫດແລວກໄດເອາໂຕລບໜ ເຈາໜາທທກຽວຂອງກຳລງຕດຕ...
419996,"ເຖງຢາງໃດກຕາມ, ທານ ອບບດ ລກອອກຈາກຫອງປະຊມທນທທສຳເລ..."
419997,ພດຕະກນຖວ ສາມາດຊວຍຫດລະດບຄເລສຕຣອລ ແລະ ປອງກນພະຍາດ...
419998,"ມນຄາກສາງມະຫາສານ,"


In [6]:
# Visualize test labels
test_labels

,label
0,ar
1,ar
2,ar
3,ar
4,ar
...,...
419995,lo
419996,lo
419997,lo
419998,lo


In [7]:
# Shuffle training data and visualize after shuffling
train_df = train_df.sample(frac=1, random_state=69).reset_index(drop=True)
train_df

,text,label
0,"Im Cafe Kaos, beim Verein Lebensbrücke in der ...",de
1,Об этом сообщила на своей странице в Facebook ...,ru
2,ສະບບເລກທ 2088 Kejie ຖະຫນນ Qingshan ອດສາຫະກາ Pa...,lo
3,"After a sell-out event last year, this much-lo...",en
4,"В Америке в связи с тем, что там распространен...",ru
...,...,...
1679995,"For the past three years, Netanyahu has bet he...",en
1679996,Sensing a bit of a casual vibe in Australia’s ...,en
1679997,But it should mean demonstrating excellence of...,en
1679998,Les figurines en vinyle les plus célèbres du m...,fr


In [8]:
# Vectorize the text using TF-IDF
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1,3), max_features=50000)

X_train = vectorizer.fit_transform(train_df['text'])
X_test = vectorizer.transform(test_df['text'])

y_train = train_df['label']
y_test = test_labels['label']

In [9]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1680000, 50000), (420000, 50000), (1680000,), (420000,))

In [10]:
X_train, X_test

(<Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 321427586 stored elements and shape (1680000, 50000)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 80382332 stored elements and shape (420000, 50000)>)

In [11]:
# Train the model with SVM
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [12]:
# Making predictions
y_pred = nb_model.predict(X_test)

In [13]:
# Evaluating model
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)
print('Classification Report:')
print(classification_report(y_test, y_pred))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.9991309523809524
Classification Report:
              precision    recall  f1-score   support

          ar       1.00      1.00      1.00     60000
          de       1.00      1.00      1.00     60000
          en       1.00      1.00      1.00     60000
          fr       1.00      1.00      1.00     60000
          lo       1.00      1.00      1.00     60000
          ru       1.00      1.00      1.00     60000
          vi       1.00      1.00      1.00     60000

    accuracy                           1.00    420000
   macro avg       1.00      1.00      1.00    420000
weighted avg       1.00      1.00      1.00    420000

Confusion Matrix:
[[59999     0     0     1     0     0     0]
 [    0 59879   114     7     0     0     0]
 [    0     7 59978    14     0     1     0]
 [    0    24   131 59845     0     0     0]
 [    0     5    32     1 59960     0     2]
 [    0     0     1     0     0 59999     0]
 [    0     2    23     0     0     0 59975]]


In [ ]:
# Test some sentences
sentences = [
    "Bonjour, comment ça va ?",        
    "Xin chào, bạn khỏe không?",       
    "Hello, how are you?",             
    "Привет, как дела?",               
    "Guten Tag, wie geht es Ihnen?",   
    "مرحبا، كيف حالك؟",                 
    "ສະບາຍດີ, ເຈົ້າສະບາຍດີບໍ?" 
]

# Transform sentences to TF-IDF vectors
X_new = vectorizer.transform(sentences)

# Predict labels
predictions = nb_model.predict(X_new)

# Show results with predicted labels
for sent, pred in zip(sentences, predictions):
    print(f"Input: {sent}\nPredicted language: {pred}\n")

Input: Bonjour, comment ça va ?
Predicted language: fr

Input: Xin chào, bạn khỏe không?
Predicted language: vi

Input: Hello, how are you?
Predicted language: en

Input: Привет, как дела?
Predicted language: ru

Input: Guten Tag, wie geht es Ihnen?
Predicted language: de

Input: مرحبا، كيف حالك؟
Predicted language: ar

Input: ສະບາຍດີ, ເຈົ້າສະບາຍດີບໍ?
Predicted language: lo

